<a href="https://colab.research.google.com/github/jhjh5823/Crawling/blob/master/%ED%95%9C%EB%B9%9B%EC%B6%9C%ED%8C%90%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm

In [2]:
base_url='https://www.hanbit.co.kr/'
sub_url='store/books/new_book_list.html'
url = base_url+sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [3]:
lis = soup.select('.sub_book_list')

In [4]:
li = lis[0]
href = li.find('a')['href']

* sub page에서 정보를 가져옴

In [5]:
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

* 제목 가져오기

In [6]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'구글 앱스 스크립트 완벽 가이드'

In [7]:
book_list = info.select('.info_list > li')
author = book_list[0].find('span').get_text().strip()
author

'다카하시 노리아키'

In [8]:
if len(book_list) >= 5:
  translator = book_list[1].find('span').get_text().strip().split(',')[0]
else:
  translator = book_list[1].find('span').get_text().strip()

translator

'김모세'

* 날짜

In [9]:
date = book_list[2].find('span').get_text().strip()
date
book_list

[<li><strong>저자 : </strong><span>다카하시 노리아키 </span></li>,
 <li><strong>번역 : </strong><span>김모세 </span></li>,
 <li><strong>출간 : </strong><span>2022-07-05</span></li>,
 <li><strong>페이지 : </strong><span>696 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791169210010</span></li>,
 <li><strong>물류코드 :</strong><span>11001</span></li>]

* 페이지

In [10]:
page = book_list[3].find('span').get_text().strip().split()[0]
page = int(page)
page

696

In [11]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales = int(re.sub('[^0-9]', '', sales))

In [ ]:
lines = []
for page in tqdm(range(1,11)):
  url = f'{base_url}{sub_url}?page={page}'
  res = requests.get(url)
  soup = BeautifulSoup(res.text, 'html.parser')
  lis = soup.select('.sub_book_list')

  for li in lis:
      book_url = li.find('a')['href']
      res = requests.get(base_url + book_url)
      book_soup = BeautifulSoup(res.text, 'html.parser')

      info = book_soup.select_one('.store_product_info_box')
      title = info.find('h3').string
      book_lis = info.select('.info_list > li')
      author = book_lis[0].find('span').get_text().strip()
      if len(book_lis) == 5:
          translator = '-'
      else:
          translator = book_lis[1].find('span').get_text().strip()
      date = book_lis[-4].find('span').get_text().strip()
      page = book_lis[-3].find('span').get_text().strip().split()[0]
      
      payments = book_soup.select_one('.payment_box.curr').find_all('p')
      sales = payments[1].select_one('.pbr').get_text().split('(')[0]
      sales = int(re.sub('[^0-9]','',sales))

      lines.append([title,author,translator,date,page,sales])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df

In [ ]:
df.to_csv('한빛신규도서.csv', index=False)